In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
import re

def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    minute  = int(x[10:12])
    sec  = int(x[12:])
    return dt.datetime(year, month, day, hour,minute)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)
    
def string2num2(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    return int(x)

PATH = "Jupyter Temp/lg 공모전/data/"

In [2]:
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_qual  = pd.read_csv(PATH+'train_quality_data.csv')
train_prob  = pd.read_csv(PATH+'train_problem_data.csv')

C:\Users\sue_b\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_qual  = pd.read_csv(PATH+'test_quality_data.csv')

C:\Users\sue_b\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
model_train = pd.DataFrame(data={"user_id": train_err["user_id"].unique()})

In [7]:
model_train.sort_values(by="user_id",inplace=True)
model_train.head()

,user_id
0,10000
1,10001
2,10002
3,10003
4,10004


In [8]:
#모델 개수
model_train = model_train.merge(train_err.groupby(by="user_id")["model_nm"].nunique().reset_index(),how="left",on="user_id")
model_train.head()

,user_id,model_nm
0,10000,1
1,10001,1
2,10002,1
3,10003,1
4,10004,1


In [9]:
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [10]:
def model6(x):
    if x == "8.5.3":
        result = "08.5.3"
    elif x == "10":
        result = "10.00.00"
    else:
        result = x
    return result

In [11]:
train_err["fwver"] = train_err["fwver"].apply(lambda x : model6(x))

In [12]:
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [13]:
temp = train_err.groupby(by=["user_id","model_nm"])["fwver"].nunique().reset_index()

In [14]:
temp["update"] = temp["fwver"].apply(lambda x : x-1)

In [15]:
temp = temp.groupby(by="user_id")["update"].sum().reset_index()

In [16]:
temp.head()

,user_id,update
0,10000,0
1,10001,1
2,10002,0
3,10003,1
4,10004,1


In [17]:
model_train = model_train.merge(temp,how="left",on="user_id")

In [18]:
model_train.head()

,user_id,model_nm,update
0,10000,1,0
1,10001,1,1
2,10002,1,0
3,10003,1,1
4,10004,1,1


In [19]:
display(train_err.head())
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [20]:
err_list = list(train_err["errtype"].unique())
err_list.sort()

In [21]:
len(err_list)

41

In [22]:
err_list

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42]

In [23]:
for i in range(1,43):
    model_train["err_"+str(i)] = [0] * len(model_train)

In [24]:
model_train.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_29,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42
0,10000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10001,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10002,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10003,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10004,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████| 16554663/16554663 [01:50<00:00, 149625.78it/s]


(15000, 42)

In [26]:
error[:2]

array([[  0.,   0.,   8., 104.,   0.,   1.,   1.,   0.,   0.,   7.,  15.,
         16.,   1.,  10.,  59.,  61.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,  32.,   1.,   1.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,  53.,   1.,   1.,   0.,   0.,   0.,  10.,
         11.,   1.,  15., 151., 128.,   0.,   4.,   1.,   3.,   1., 756.,
        751.,   5.,   1.,  22.,   0.,   0.,   0.,   0., 250.,   0.,  10.,
         18.,   0.,   1.,   1.,   0.,   0., 113.,  56.,   1.]])

In [27]:
error_tp = error.transpose()
error_tp.shape

(42, 15000)

In [28]:
error_tp[:4]

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  8.,   0.,   2., ...,   0.,   0.,   4.],
       [104.,   0., 132., ...,   1.,   0., 192.]])

In [29]:
for i in range(len(error_tp)):
    model_train[model_train.columns[i+3]] = error_tp[i]   

In [30]:
model_train.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_29,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42
0,10000,1,0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,1,1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
2,10002,1,0,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,1,1,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
4,10004,1,1,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0


In [31]:
del model_train["err_29"]

In [32]:
fw = train_err.groupby("model_nm")["fwver"].unique()

In [33]:
fw

model_nm
model_0    [04.22.1750, 04.22.1778, 04.22.1684, 04.22.166...
model_1    [04.16.3553, 04.16.3571, 04.16.3439, 04.16.356...
model_2    [04.33.1185, 04.33.1261, 04.33.1149, 04.33.117...
model_3    [05.15.2138, 05.15.2120, 05.15.2090, 05.15.310...
model_4                 [03.11.1149, 03.11.1167, 03.11.1141]
model_5                 [04.82.1684, 04.82.1778, 04.82.1730]
model_6                                   [10.00.00, 08.5.3]
model_7                             [05.66.3237, 05.66.3571]
model_8                             [04.73.2237, 04.73.2571]
Name: fwver, dtype: object

In [34]:
fw_dic = {}
for mod in fw:
    mod = mod.sort()

fw

model_nm
model_0    [04.22.1442, 04.22.1656, 04.22.1666, 04.22.168...
model_1    [04.16.2641, 04.16.3345, 04.16.3439, 04.16.355...
model_2    [04.33.1095, 04.33.1125, 04.33.1149, 04.33.117...
model_3    [05.15.2090, 05.15.2092, 05.15.2114, 05.15.212...
model_4                 [03.11.1141, 03.11.1149, 03.11.1167]
model_5                 [04.82.1684, 04.82.1730, 04.82.1778]
model_6                                   [08.5.3, 10.00.00]
model_7                             [05.66.3237, 05.66.3571]
model_8                             [04.73.2237, 04.73.2571]
Name: fwver, dtype: object

In [35]:
for i in fw:
    print(i)

['04.22.1442' '04.22.1656' '04.22.1666' '04.22.1684' '04.22.1750'
 '04.22.1778']
['04.16.2641' '04.16.3345' '04.16.3439' '04.16.3553' '04.16.3569'
 '04.16.3571']
['04.33.1095' '04.33.1125' '04.33.1149' '04.33.1171' '04.33.1185'
 '04.33.1261']
['05.15.2090' '05.15.2092' '05.15.2114' '05.15.2120' '05.15.2122'
 '05.15.2138' '05.15.3104']
['03.11.1141' '03.11.1149' '03.11.1167']
['04.82.1684' '04.82.1730' '04.82.1778']
['08.5.3' '10.00.00']
['05.66.3237' '05.66.3571']
['04.73.2237' '04.73.2571']


In [36]:
for i in fw:
    for j in range(len(i)):
        fw_dic[i[j]] = [j,len(i)]

In [37]:
fw_dic

{'04.22.1442': [0, 6],
 '04.22.1656': [1, 6],
 '04.22.1666': [2, 6],
 '04.22.1684': [3, 6],
 '04.22.1750': [4, 6],
 '04.22.1778': [5, 6],
 '04.16.2641': [0, 6],
 '04.16.3345': [1, 6],
 '04.16.3439': [2, 6],
 '04.16.3553': [3, 6],
 '04.16.3569': [4, 6],
 '04.16.3571': [5, 6],
 '04.33.1095': [0, 6],
 '04.33.1125': [1, 6],
 '04.33.1149': [2, 6],
 '04.33.1171': [3, 6],
 '04.33.1185': [4, 6],
 '04.33.1261': [5, 6],
 '05.15.2090': [0, 7],
 '05.15.2092': [1, 7],
 '05.15.2114': [2, 7],
 '05.15.2120': [3, 7],
 '05.15.2122': [4, 7],
 '05.15.2138': [5, 7],
 '05.15.3104': [6, 7],
 '03.11.1141': [0, 3],
 '03.11.1149': [1, 3],
 '03.11.1167': [2, 3],
 '04.82.1684': [0, 3],
 '04.82.1730': [1, 3],
 '04.82.1778': [2, 3],
 '08.5.3': [0, 2],
 '10.00.00': [1, 2],
 '05.66.3237': [0, 2],
 '05.66.3571': [1, 2],
 '04.73.2237': [0, 2],
 '04.73.2571': [1, 2]}

In [38]:
def fw_trans(x):
    return(float(fw_dic[x][0]) / (int(fw_dic[x][1]) - 1))

In [39]:
train_err["fw_rank"] = train_err["fwver"].apply(lambda x: fw_trans(x))
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode,fw_rank
0,10000,20201101025616,model_3,05.15.2138,15,1,0.833333
1,10000,20201101030309,model_3,05.15.2138,12,1,0.833333
2,10000,20201101030309,model_3,05.15.2138,11,1,0.833333
3,10000,20201101050514,model_3,05.15.2138,16,1,0.833333
4,10000,20201101050515,model_3,05.15.2138,4,0,0.833333


In [40]:
fwrank_temp = train_err.groupby(by=["user_id","model_nm"])["fw_rank"].mean().reset_index()

In [41]:
fwrank_temp.head()

,user_id,model_nm,fw_rank
0,10000,model_3,0.833333
1,10001,model_2,0.967949
2,10002,model_3,0.833333
3,10003,model_2,0.947059
4,10004,model_0,0.833977


In [42]:
fwrank_temp = fwrank_temp.groupby(by="user_id")["fw_rank"].mean().reset_index()
fwrank_temp.head()

,user_id,fw_rank
0,10000,0.833333
1,10001,0.967949
2,10002,0.833333
3,10003,0.947059
4,10004,0.833977


In [43]:
len(fwrank_temp)

15000

In [44]:
model_train = model_train.merge(fwrank_temp,how="left",on="user_id")

In [45]:
model_train.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,fw_rank
0,10000,1,0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333
1,10001,1,1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.967949
2,10002,1,0,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333
3,10003,1,1,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.947059
4,10004,1,1,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0.833977


In [46]:
train_qual.head()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [47]:
for i in train_qual.columns[3:]:
    train_qual[i] = train_qual[i].apply(lambda x: string2num(x))
    train_qual[i] = train_qual[i].apply(lambda x: 0 if x == -1 else x)

In [48]:
train_qual.dtypes

time           int64
user_id        int64
fwver         object
quality_0      int64
quality_1      int64
quality_2      int64
quality_3      int64
quality_4      int64
quality_5      int64
quality_6      int64
quality_7      int64
quality_8      int64
quality_9      int64
quality_10     int64
quality_11     int64
quality_12     int64
dtype: object

In [49]:
train_qual_1 = train_qual.groupby(by=["time","user_id"]).mean().reset_index()

In [50]:
train_qual_1.head()

,time,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201031235000,10136,1.666667,0.166667,1.666667,0.0,0.0,0.333333,0.166667,0.0,0.0,0.0,2.0,0.166667,0.0
1,20201031235000,12060,1.666667,0.166667,1.666667,0.0,0.0,0.666667,0.166667,0.0,0.0,0.0,6.0,0.166667,0.0
2,20201031235000,12729,2.500000,0.583333,2.500000,0.0,0.0,0.250000,0.250000,0.0,4.0,0.0,0.0,0.250000,0.0
3,20201031235000,13495,3.333333,0.333333,3.333333,0.0,0.0,1.333333,0.333333,0.0,0.0,0.0,12.0,0.333333,0.0
4,20201031235000,13923,0.000000,0.000000,0.000000,0.0,0.0,0.583333,1.166667,14.0,0.0,0.0,7.0,0.000000,0.0


In [51]:
train_qual_max = train_qual_1.groupby(by="user_id").max().reset_index()
train_qual_max.head()

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,20201130210000,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.000000,0.0,0.0,0.0,8.0,0.000000,0.0
1,10002,20201130030000,1.666667,0.166667,1.666667,0.0,0.0,0.666667,3.666667,44.0,0.0,1.0,8.0,0.166667,0.0
2,10004,20201104123000,1.666667,0.166667,1.666667,0.0,0.0,0.416667,7.250000,87.0,0.0,0.0,3.0,0.166667,0.0
3,10005,20201119040000,8.333333,0.833333,8.333333,0.0,0.0,1.166667,3.833333,36.0,0.0,0.0,6.0,0.833333,0.0
4,10006,20201127030000,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.333333,4.0,0.0,0.0,4.0,0.000000,0.0


In [52]:
for i in range(13):
    train_qual_max.rename(columns= {"quality_"+str(i) : "q_meanmax_"+str(i)},inplace=True)

train_qual_max.head()

,user_id,time,q_meanmax_0,q_meanmax_1,q_meanmax_2,q_meanmax_3,q_meanmax_4,q_meanmax_5,q_meanmax_6,q_meanmax_7,q_meanmax_8,q_meanmax_9,q_meanmax_10,q_meanmax_11,q_meanmax_12
0,10000,20201130210000,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.000000,0.0,0.0,0.0,8.0,0.000000,0.0
1,10002,20201130030000,1.666667,0.166667,1.666667,0.0,0.0,0.666667,3.666667,44.0,0.0,1.0,8.0,0.166667,0.0
2,10004,20201104123000,1.666667,0.166667,1.666667,0.0,0.0,0.416667,7.250000,87.0,0.0,0.0,3.0,0.166667,0.0
3,10005,20201119040000,8.333333,0.833333,8.333333,0.0,0.0,1.166667,3.833333,36.0,0.0,0.0,6.0,0.833333,0.0
4,10006,20201127030000,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.333333,4.0,0.0,0.0,4.0,0.000000,0.0


In [53]:
len(train_qual_max)

8281

In [54]:
del train_qual_max["time"]

In [55]:
model_train = model_train.merge(train_qual_max,how="left",on="user_id")

In [56]:
model_train.isna().sum()

user_id            0
model_nm           0
update             0
err_1              0
err_2              0
err_3              0
err_4              0
err_5              0
err_6              0
err_7              0
err_8              0
err_9              0
err_10             0
err_11             0
err_12             0
err_13             0
err_14             0
err_15             0
err_16             0
err_17             0
err_18             0
err_19             0
err_20             0
err_21             0
err_22             0
err_23             0
err_24             0
err_25             0
err_26             0
err_27             0
err_28             0
err_30             0
err_31             0
err_32             0
err_33             0
err_34             0
err_35             0
err_36             0
err_37             0
err_38             0
err_39             0
err_40             0
err_41             0
err_42             0
fw_rank            0
q_meanmax_0     6719
q_meanmax_1     6719
q_meanmax_2  

In [57]:
model_train.fillna(0,inplace=True)

In [58]:
train_qual_2 = train_qual.groupby(by=["user_id","time"]).nunique().reset_index()
train_qual_2.head()

,user_id,time,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,20201129090000,1,1,1,1,1,1,2,1,1,1,1,1,1,1
1,10000,20201130210000,1,1,1,1,1,1,2,1,1,1,1,1,1,1
2,10002,20201104110000,1,2,1,2,1,1,1,1,1,1,1,1,1,1
3,10002,20201106010000,1,1,1,1,1,1,2,1,1,1,1,1,1,1
4,10002,20201111010000,1,1,1,1,1,1,3,2,1,1,1,1,1,1


In [59]:
for i in train_qual_2.columns[3:]:
    train_qual_2[i] = train_qual_2[i].apply(lambda x: x-1)
train_qual_2.head()

,user_id,time,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,20201129090000,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1,10000,20201130210000,1,0,0,0,0,0,1,0,0,0,0,0,0,0
2,10002,20201104110000,1,1,0,1,0,0,0,0,0,0,0,0,0,0
3,10002,20201106010000,1,0,0,0,0,0,1,0,0,0,0,0,0,0
4,10002,20201111010000,1,0,0,0,0,0,2,1,0,0,0,0,0,0


In [60]:
train_qual_2 = train_qual_2.groupby(by="user_id").sum().reset_index()
train_qual_2.head()

,user_id,time,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,40402259300000,2,0,0,0,0,0,2,0,0,0,0,0,0,0
1,10002,161608933442000,8,2,1,2,0,0,12,3,0,0,0,0,1,0
2,10004,40402206355000,2,1,1,1,0,0,3,2,0,0,0,0,1,0
3,10005,40402238060000,2,1,1,1,0,0,3,2,0,0,0,0,1,0
4,10006,60603354277000,3,0,0,0,0,0,3,1,0,0,0,0,0,0


In [61]:
for i in range(13):
    train_qual_2.rename(columns= {"quality_"+str(i) : "q_chg_"+str(i)},inplace=True)

train_qual_2.head()

,user_id,time,fwver,q_chg_0,q_chg_1,q_chg_2,q_chg_3,q_chg_4,q_chg_5,q_chg_6,q_chg_7,q_chg_8,q_chg_9,q_chg_10,q_chg_11,q_chg_12
0,10000,40402259300000,2,0,0,0,0,0,2,0,0,0,0,0,0,0
1,10002,161608933442000,8,2,1,2,0,0,12,3,0,0,0,0,1,0
2,10004,40402206355000,2,1,1,1,0,0,3,2,0,0,0,0,1,0
3,10005,40402238060000,2,1,1,1,0,0,3,2,0,0,0,0,1,0
4,10006,60603354277000,3,0,0,0,0,0,3,1,0,0,0,0,0,0


In [62]:
del train_qual_2["time"]
del train_qual_2["fwver"]

In [63]:
for i in [7,8,9,10,12]:
    del train_qual_2["q_chg_"+str(i)]

In [64]:
train_qual_2.head()

,user_id,q_chg_0,q_chg_1,q_chg_2,q_chg_3,q_chg_4,q_chg_5,q_chg_6,q_chg_11
0,10000,0,0,0,0,0,2,0,0
1,10002,2,1,2,0,0,12,3,1
2,10004,1,1,1,0,0,3,2,1
3,10005,1,1,1,0,0,3,2,1
4,10006,0,0,0,0,0,3,1,0


In [65]:
model_train = model_train.merge(train_qual_2,how="left",on="user_id")

In [66]:
model_train.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,fw_rank,q_meanmax_0,q_meanmax_1,q_meanmax_2,q_meanmax_3,q_meanmax_4,q_meanmax_5,q_meanmax_6,q_meanmax_7,q_meanmax_8,q_meanmax_9,q_meanmax_10,q_meanmax_11,q_meanmax_12,q_chg_0,q_chg_1,q_chg_2,q_chg_3,q_chg_4,q_chg_5,q_chg_6,q_chg_11
0,10000,1,0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.000000,0.0,0.0,0.0,8.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,10001,1,1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.967949,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10002,1,0,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,1.666667,0.166667,1.666667,0.0,0.0,0.666667,3.666667,44.0,0.0,1.0,8.0,0.166667,0.0,2.0,1.0,2.0,0.0,0.0,12.0,3.0,1.0
3,10003,1,1,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.947059,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10004,1,1,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0.833977,1.666667,0.166667,1.666667,0.0,0.0,0.416667,7.250000,87.0,0.0,0.0,3.0,0.166667,0.0,1.0,1.0,1.0,0.0,0.0,3.0,2.0,1.0


In [67]:
model_train.shape

(15000, 66)

In [68]:
train_prob.head()

,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000


In [69]:
train_prob_2 = pd.DataFrame(data = {"user_id": list(train_prob["user_id"].unique()), "label" : [1]*len(train_prob["user_id"].unique())})

In [70]:
train_prob_2.head()

,user_id,label
0,19224,1
1,23664,1
2,15166,1
3,12590,1
4,15932,1


In [71]:
model_train = model_train.merge(train_prob_2,how="left",on="user_id")

In [72]:
model_train.fillna(0,inplace=True)
model_train.head()

,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,fw_rank,q_meanmax_0,q_meanmax_1,q_meanmax_2,q_meanmax_3,q_meanmax_4,q_meanmax_5,q_meanmax_6,q_meanmax_7,q_meanmax_8,q_meanmax_9,q_meanmax_10,q_meanmax_11,q_meanmax_12,q_chg_0,q_chg_1,q_chg_2,q_chg_3,q_chg_4,q_chg_5,q_chg_6,q_chg_11,label
0,10000,1,0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.000000,0.0,0.0,0.0,8.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,10001,1,1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.967949,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,10002,1,0,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,1.666667,0.166667,1.666667,0.0,0.0,0.666667,3.666667,44.0,0.0,1.0,8.0,0.166667,0.0,2.0,1.0,2.0,0.0,0.0,12.0,3.0,1.0,0.0
3,10003,1,1,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.947059,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,1,1,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0.833977,1.666667,0.166667,1.666667,0.0,0.0,0.416667,7.250000,87.0,0.0,0.0,3.0,0.166667,0.0,1.0,1.0,1.0,0.0,0.0,3.0,2.0,1.0,1.0


In [73]:
model_train.to_csv("Jupyter Temp/lg 공모전/data/0126_train_xy.csv",index=False)